In [1]:
!pip install xgboost

In [2]:
!pip install optuna

     |████████████████████████████████| 308 kB 5.0 MB/s 
     |████████████████████████████████| 81 kB 11.6 MB/s 
     |████████████████████████████████| 210 kB 85.6 MB/s 
     |████████████████████████████████| 75 kB 6.0 MB/s 
     |████████████████████████████████| 150 kB 78.6 MB/s 
     |████████████████████████████████| 49 kB 7.7 MB/s 
     |████████████████████████████████| 113 kB 55.6 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=430f107d1dfd877e8476e8ded40ca46b3a558e08036e0f3ee8a947cc5f4bf61b
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [3]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, accuracy_score

In [4]:
import os, random # random seed 고정 
def set_seeds(seed): 
    os.environ['PYTHONHASHSEED'] = str(seed) 
    random.seed(seed) 
    np.random.seed(seed) 
    # tf.random.set_seed(seed) # Tensorflow 사용시 
    
SEED = 555
set_seeds(SEED)

In [5]:
cd ./drive/MyDrive/미니_프로젝트/data

/content/drive/MyDrive/미니_프로젝트/data


# 모든 feature가 다 포함

In [6]:
train_fast = pd.read_csv('./train_fast_final.csv')
test_fast = pd.read_csv('./test_fast_final.csv')
train_slow = pd.read_csv('./train_slow_final.csv')
test_slow = pd.read_csv('./test_slow_final.csv')

In [7]:
train_fast.shape, test_fast.shape, train_slow.shape, test_slow.shape

((102316, 68), (25575, 68), (102294, 68), (25575, 68))

## 급속 충전기

In [8]:
X_fast_train = train_fast.drop(['fast_exist'], axis=1)
y_fast_train = train_fast.fast_exist

X_fast_test = test_fast.drop(['fast_exist'], axis=1)
y_fast_test = test_fast.fast_exist

### XGBoost

In [ ]:
XGB.fit(X_fast_train, y_fast_train)

XGBClassifier(interaction_constraints='', learning_rate=0.300000012,
              max_depth=6, n_jobs=-1, num_parallel_tree=1, tree_method='exact',
              validate_parameters=1)

In [18]:
def XGB_objective(trial):
    base_score = 0.5
    booster = trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart'])
    colsample_bylevel = trial.suggest_float('colsample_bylevel', 0, 1)
    colsample_bynode = trial.suggest_float('colsample_bynode', 0, 1)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0, 1)
    gamma = trial.suggest_float('gamma', 0, 10)
    importance_type = trial.suggest_categorical('importance_type', ['weight', 'gain', 'cover', 'total_gain', 'total_cover'])
    learning_rate = trial.suggest_float('learning_rate', 0, 1)
    max_delta_step = trial.suggest_float('max_delta_step', 0, 10)
    max_depth = trial.suggest_int('max_depth', 1, 10)
    min_child_weight = trial.suggest_float('min_child_weight', 0, 10)
    n_estimators =  trial.suggest_int('n_estimators', 100, 500)
    n_jobs = -1
    num_parallel_tree = trial.suggest_int('num_parallel_tree', 1,10)
    random_state = 42
    reg_alpha=trial.suggest_float('reg_alpha', 0, 10)
    reg_lambda=trial.suggest_float('reg_lambda', 0, 10)
    scale_pos_weight=1
    subsample=trial.suggest_float('subsample', 0, 1)
    tree_method='gpu_hist'
    validate_parameters=1
    
   
    model = XGBClassifier(base_score=base_score, booster=booster, colsample_bylevel=colsample_bylevel, colsample_bytree=colsample_bytree, colsample_bynode=colsample_bynode,
                          gamma=gamma, importance_type=importance_type, learning_rate=learning_rate, max_delta_step=max_delta_step, max_depth=max_depth, min_child_weight=min_child_weight,
                          n_estimators=n_estimators, n_jobs=n_jobs, num_parallel_tree=num_parallel_tree, random_state=random_state, reg_alpha=reg_alpha, reg_lambda=reg_lambda,
                          scale_pos_weight=scale_pos_weight, subsample=subsample, tree_method=tree_method, validate_parameters=validate_parameters
                          )
    print('=============================================')
    print(model)
    model.fit(X_fast_train, y_fast_train)
    print('피팅 완료')
    y_fast_prob = model.predict_proba(X_fast_test)
    print('확률값 도출 완료')
    for threshold in range(1,10):
        threshold = round(threshold*0.1, 1)
        y_fast_pred = np.where(y_fast_prob>=threshold, 1, 0)
        print(f'{threshold} :{f1_score(y_fast_test, y_fast_pred[:,1])}')
    print('threshold당 성능 확인')

    y_fast_pred = model.predict(X_fast_test)
    print('예측 완료')
    f1 = f1_score(y_fast_test, y_fast_pred)
    print('f1 score 도출 완료: ', f1)

    return f1
    
#Execute optuna and set hyperparameters
XGB_study = optuna.create_study(direction='maximize')
XGB_study.optimize(XGB_objective, n_trials=100)


#Create an instance with tuned hyperparameters
optimized_XGB = XGBClassifier(base_score=0.5, booster=XGB_study.best_params['booster'], colsample_bylevel=XGB_study.best_params['colsample_bylevel'], colsample_bytree=XGB_study.best_params['colsample_bytree'], 
                              colsample_bynode=XGB_study.best_params['colsample_bynode'], gamma=XGB_study.best_params['gamma'], importance_type=XGB_study.best_params['importance_type'], learning_rate=XGB_study.best_params['learning_rate'], 
                              max_delta_step=XGB_study.best_params['max_delta_step'], max_depth=XGB_study.best_params['max_depth'], min_child_weight=XGB_study.best_params['min_child_weight'], n_estimators=XGB_study.best_params['n_estimators'], 
                              n_jobs=-1, num_parallel_tree=XGB_study.best_params['num_parallel_tree'], random_state=42, reg_alpha=0, 
                              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='gpu_hist', 
                              validate_parameters=1
                              )

[I 2022-02-25 14:48:22,832] A new study created in memory with name: no-name-7311c59a-bfa7-47cc-a133-ea281acf08be


XGBClassifier(booster='dart', colsample_bylevel=0.23718505025869385,
              colsample_bynode=0.5165783794515452,
              colsample_bytree=0.43938055430535794, gamma=1.331757807954821,
              learning_rate=0.0017546114598654405,
              max_delta_step=1.5408854247168136, max_depth=9,
              min_child_weight=1.3886964908665844, n_estimators=221, n_jobs=-1,
              num_parallel_tree=4, random_state=42, reg_alpha=6.156973185645077,
              reg_lambda=0.39746996082684394, subsample=0.3300321142414372,
              tree_method='gpu_hist', validate_parameters=1)
피팅 완료
확률값 도출 완료
0.1 :0.001953125
0.2 :0.001953125
0.3 :0.001953125
0.4 :0.04191114836546521
0.5 :0.10087719298245613
0.6 :0.3125
0.7 :0.0
0.8 :0.0
0.9 :0.0
threshold당 성능 확인


[I 2022-02-25 14:52:01,727] Trial 0 finished with value: 0.10087719298245613 and parameters: {'booster': 'dart', 'colsample_bylevel': 0.23718505025869385, 'colsample_bynode': 0.5165783794515452, 'colsample_bytree': 0.43938055430535794, 'gamma': 1.331757807954821, 'importance_type': 'cover', 'learning_rate': 0.0017546114598654405, 'max_delta_step': 1.5408854247168136, 'max_depth': 9, 'min_child_weight': 1.3886964908665844, 'n_estimators': 221, 'num_parallel_tree': 4, 'reg_alpha': 6.156973185645077, 'reg_lambda': 0.39746996082684394, 'subsample': 0.3300321142414372}. Best is trial 0 with value: 0.10087719298245613.


예측 완료
f1 score 도출 완료:  0.10087719298245613
XGBClassifier(colsample_bylevel=0.37142140470781115,
              colsample_bynode=0.30061868157996285,
              colsample_bytree=0.5991976122900746, gamma=8.29366882083073,
              learning_rate=0.7932861928931114,
              max_delta_step=2.213336617882138, max_depth=1,
              min_child_weight=9.595746478122356, n_estimators=390, n_jobs=-1,
              num_parallel_tree=5, random_state=42, reg_alpha=9.553935101224624,
              reg_lambda=5.783028907883749, subsample=0.8322804283126175,
              tree_method='gpu_hist', validate_parameters=1)


[I 2022-02-25 14:52:08,291] Trial 1 finished with value: 0.31007751937984496 and parameters: {'booster': 'gbtree', 'colsample_bylevel': 0.37142140470781115, 'colsample_bynode': 0.30061868157996285, 'colsample_bytree': 0.5991976122900746, 'gamma': 8.29366882083073, 'importance_type': 'weight', 'learning_rate': 0.7932861928931114, 'max_delta_step': 2.213336617882138, 'max_depth': 1, 'min_child_weight': 9.595746478122356, 'n_estimators': 390, 'num_parallel_tree': 5, 'reg_alpha': 9.553935101224624, 'reg_lambda': 5.783028907883749, 'subsample': 0.8322804283126175}. Best is trial 1 with value: 0.31007751937984496.


...

[I 2022-02-25 17:57:48,752] Trial 66 finished with value: 0.631578947368421 and parameters: {'booster': 'dart', 'colsample_bylevel': 0.3296615716642682, 'colsample_bynode': 0.9099397862186192, 'colsample_bytree': 0.6882679635237732, 'gamma': 6.621903886817292, 'importance_type': 'total_gain', 'learning_rate': 0.19903424025167887, 'max_delta_step': 4.403546793152496, 'max_depth': 6, 'min_child_weight': 0.4816202289225182, 'n_estimators': 319, 'num_parallel_tree': 2, 'reg_alpha': 2.902115596384586, 'reg_lambda': 0.19524343964539315, 'subsample': 0.9002961074233268}. Best is trial 63 with value: 0.7450980392156863.


예측 완료
f1 score 도출 완료:  0.631578947368421
XGBClassifier(booster='dart', colsample_bylevel=0.4513228204026503,
              colsample_bynode=0.8627000890138542,
              colsample_bytree=0.734597976670955, gamma=7.560109986978303,
              learning_rate=0.3044498469622904, max_delta_step=4.23495322050842,
              max_depth=7, min_child_weight=1.5907177826970136,
              n_estimators=261, n_jobs=-1, num_parallel_tree=2, random_state=42,
              reg_alpha=0.9555820450598402, reg_lambda=0.6083685337681632,
              subsample=0.7365425305888578, tree_method='gpu_hist',
              validate_parameters=1)
피팅 완료
확률값 도출 완료
0.1 :0.5384615384615384
0.2 :0.6086956521739131
0.3 :0.6774193548387097
0.4 :0.6909090909090909
0.5 :0.7058823529411765
0.6 :0.72
0.7 :0.6938775510204083
0.8 :0.6938775510204083
0.9 :0.6222222222222222
threshold당 성능 확인


[I 2022-02-25 17:59:26,132] Trial 67 finished with value: 0.7058823529411765 and parameters: {'booster': 'dart', 'colsample_bylevel': 0.4513228204026503, 'colsample_bynode': 0.8627000890138542, 'colsample_bytree': 0.734597976670955, 'gamma': 7.560109986978303, 'importance_type': 'total_gain', 'learning_rate': 0.3044498469622904, 'max_delta_step': 4.23495322050842, 'max_depth': 7, 'min_child_weight': 1.5907177826970136, 'n_estimators': 261, 'num_parallel_tree': 2, 'reg_alpha': 0.9555820450598402, 'reg_lambda': 0.6083685337681632, 'subsample': 0.7365425305888578}. Best is trial 63 with value: 0.7450980392156863.


예측 완료
f1 score 도출 완료:  0.7058823529411765
XGBClassifier(booster='dart', colsample_bylevel=0.4681805232590638,
              colsample_bynode=0.8560195049460995,
              colsample_bytree=0.7245936893277457, gamma=7.6072680066253096,
              learning_rate=0.26500276402605644,
              max_delta_step=5.105599852023872, max_depth=8,
              min_child_weight=1.8929390457183244, n_estimators=260, n_jobs=-1,
              num_parallel_tree=3, random_state=42,
              reg_alpha=0.9884589377952011, reg_lambda=0.8403097227589081,
              subsample=0.667142065635105, tree_method='gpu_hist',
              validate_parameters=1)


KeyboardInterrupt: ignored

XGBClassifier(booster='dart', colsample_bylevel=0.3928152565945655,
              colsample_bynode=0.8870570346704255,
              colsample_bytree=0.8290834140669368, gamma=7.13467750459079,
              learning_rate=0.18898804856733348,
              max_delta_step=4.187483786962758, max_depth=7,
              min_child_weight=0.6730716133375483, n_estimators=337, n_jobs=-1,
              num_parallel_tree=2, random_state=42,
              reg_alpha=1.7004811519199183, reg_lambda=0.41095413156908833,
              subsample=0.8838964398570223, tree_method='gpu_hist',
              validate_parameters=1)
피팅 완료
확률값 도출 완료
0.1 :0.5365853658536586
0.2 :0.6176470588235294
0.3 :0.6229508196721312
0.4 :0.7037037037037037
0.5 :0.7450980392156863
0.6 :0.72
0.7 :0.6666666666666666
0.8 :0.6521739130434783

In [13]:
params = {'booster': 'dart', 'colsample_bylevel': 0.5915486982189424, 'colsample_bynode': 0.5716410777785312, 'colsample_bytree': 0.4854852391096862, 'gamma': 2.247085034165828, 'importance_type': 'gain', 'learning_rate': 0.11318511181638041, 'max_delta_step': 1.5998053901431368, 'max_depth': 8, 'min_child_weight': 0.4117212664636557, 'n_estimators': 462, 'num_parallel_tree': 5, 'reg_alpha': 0.0015988125738429504, 'reg_lambda': 6.762981527776633, 'subsample': 0.8684584879991994}

In [14]:
optimized_XGB = XGBClassifier(**params)

In [ ]:
# optuna.visualization.plot_param_importances(XGB_study)

NameError: ignored

In [ ]:
# optuna.visualization.plot_optimization_history(XGB_study)

NameError: ignored

In [ ]:
optimized_XGB.fit(X_fast_train, y_fast_train)

XGBClassifier(booster='dart', colsample_bylevel=0.5915486982189424,
              colsample_bynode=0.5716410777785312,
              colsample_bytree=0.4854852391096862, gamma=2.247085034165828,
              learning_rate=0.11318511181638041,
              max_delta_step=1.5998053901431368, max_depth=8,
              min_child_weight=0.4117212664636557, n_estimators=462,
              num_parallel_tree=5, reg_alpha=0.0015988125738429504,
              reg_lambda=6.762981527776633, subsample=0.8684584879991994)

In [ ]:
y_fast_prob = optimized_XGB.predict_proba(X_fast_test)

In [ ]:
y_fast_pred = optimized_XGB.predict(X_fast_test)
f1_score(y_fast_test, y_fast_pred)

0.6129032258064517

In [ ]:
for threshold in range(1,10):
    threshold = round(threshold*0.1, 1)
    y_fast_pred = np.where(y_fast_prob>=threshold, 1, 0)
    print(f'{threshold} :{f1_score(y_fast_test, y_fast_pred[:,1])}')

0.1 :0.37168141592920356
0.2 :0.43956043956043955
0.3 :0.5000000000000001
0.4 :0.5428571428571428
0.5 :0.6129032258064517
0.6 :0.631578947368421
0.7 :0.7058823529411765
0.8 :0.6382978723404256
0.9 :0.6382978723404256


In [ ]:
# XGB = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
#               colsample_bynode=1, colsample_bytree=1, gamma=0,
#               importance_type='gain', interaction_constraints='',
#               learning_rate=0.300000012, max_delta_step=0, max_depth=6,
#               min_child_weight=1,
#               n_estimators=100, n_jobs=-1, num_parallel_tree=1, random_state=42,
#               reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
#               tree_method='exact', validate_parameters=1)

In [ ]:
# XGB.fit(X_fast_train, y_fast_train)

XGBClassifier(interaction_constraints='', learning_rate=0.300000012,
              max_depth=6, n_jobs=-1, num_parallel_tree=1, random_state=42,
              tree_method='exact', validate_parameters=1)

In [ ]:
# y_fast_prob = XGB.predict_proba(X_fast_test)

In [ ]:
# for threshold in range(1,10):
#     threshold = round(threshold*0.1, 1)
#     y_fast_pred = np.where(y_fast_prob>=threshold, 1, 0)
#     print(f'{threshold} :{f1_score(y_fast_test, y_fast_pred[:,1])}')

0.1 :0.6666666666666667
0.2 :0.7241379310344828
0.3 :0.7272727272727272
0.4 :0.7407407407407408
0.5 :0.7058823529411765
0.6 :0.68
0.7 :0.6530612244897959
0.8 :0.6521739130434783
0.9 :0.6363636363636364


* **오버피팅!!!!!!!!!**

In [ ]:
# y_fast_pred_train = XGB.predict(X_fast_train)
# f1_score(y_fast_train, y_fast_pred_train)

1.0

## 완속 충전기

In [ ]:
X_slow_train = train_slow.drop(['slow_exist'], axis=1)
y_slow_train = train_slow.slow_exist

X_slow_test = test_slow.drop(['slow_exist'], axis=1)
y_slow_test = test_slow.slow_exist

### XGBoost

In [ ]:
def XGB_objective(trial):
    base_score = 0.5
    booster = trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart'])
    colsample_bylevel = trial.suggest_float('colsample_bylevel', 0, 1)
    colsample_bynode = trial.suggest_float('colsample_bynode', 0, 1)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0, 1)
    gamma = trial.suggest_float('gamma', 0, 10)
    importance_type = trial.suggest_categorical('importance_type', ['weight', 'gain', 'cover', 'total_gain', 'total_cover'])
    learning_rate = trial.suggest_float('learning_rate', 0, 1)
    max_delta_step = trial.suggest_float('max_delta_step', 0, 10)
    max_depth = trial.suggest_int('max_depth', 1, 10)
    min_child_weight = trial.suggest_float('min_child_weight', 0, 10)
    n_estimators =  trial.suggest_int('n_estimators', 100, 500)
    n_jobs = -1
    num_parallel_tree = trial.suggest_int('num_parallel_tree', 1,10)
    random_state = 42
    reg_alpha=trial.suggest_float('reg_alpha', 0, 10)
    reg_lambda=trial.suggest_float('reg_lambda', 0, 10)
    scale_pos_weight=1
    subsample=trial.suggest_float('subsample', 0, 1)
    tree_method='gpu_hist'
    validate_parameters=1
    
   
    model = XGBClassifier(base_score=base_score, booster=booster, colsample_bylevel=colsample_bylevel, colsample_bytree=colsample_bytree, colsample_bynode=colsample_bynode,
                          gamma=gamma, importance_type=importance_type, learning_rate=learning_rate, max_delta_step=max_delta_step, max_depth=max_depth, min_child_weight=min_child_weight,
                          n_estimators=n_estimators, n_jobs=n_jobs, num_parallel_tree=num_parallel_tree, random_state=random_state, reg_alpha=reg_alpha, reg_lambda=reg_lambda,
                          scale_pos_weight=scale_pos_weight, subsample=subsample, tree_method=tree_method, validate_parameters=validate_parameters
                          )

    print('=============================================')
    print(model)
    model.fit(X_slow_train, y_slow_train)
    print('피팅 완료')
    y_slow_prob = model.predict_proba(X_slow_test)
    print('확률값 도출 완료')
    for threshold in range(1,10):
        threshold = round(threshold*0.1, 1)
        y_slow_pred = np.where(y_slow_prob>=threshold, 1, 0)
        print(f'{threshold} :{f1_score(y_slow_test, y_slow_pred[:,1])}')
    print('threshold당 성능 확인')

    y_slow_pred = model.predict(X_slow_test)
    print('예측 완료')
    f1 = f1_score(y_slow_test, y_slow_pred)
    print('f1 score 도출 완료: ', f1)

    return f1
    
#Execute optuna and set hyperparameters
XGB_study_slow = optuna.create_study(direction='maximize')
XGB_study_slow.optimize(XGB_objective, n_trials=100)


#Create an instance with tuned hyperparameters
optimized_XGB_slow = XGBClassifier(base_score=0.5, booster=XGB_study_slow.best_params['booster'], colsample_bylevel=XGB_study_slow.best_params['colsample_bylevel'], 
                              colsample_bytree=XGB_study_slow.best_params['colsample_bytree'], colsample_bynode=XGB_study_slow.best_params['colsample_bynode'], gamma=XGB_study_slow.best_params['gamma'], 
                              importance_type=XGB_study_slow.best_params['importance_type'], learning_rate=XGB_study_slow.best_params['learning_rate'], 
                              max_delta_step=XGB_study_slow.best_params['max_delta_step'], max_depth=XGB_study_slow.best_params['max_depth'], min_child_weight=XGB_study_slow.best_params['min_child_weight'], 
                              n_estimators=XGB_study_slow.best_params['n_estimators'], 
                              n_jobs=-1, num_parallel_tree=XGB_study_slow.best_params['num_parallel_tree'], random_state=42, reg_alpha=0, 
                              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='gpu_hist', 
                              validate_parameters=1
                              )

[I 2022-02-25 10:59:43,019] A new study created in memory with name: no-name-d2624d34-8751-425b-96a3-0e3526dfab7e
[I 2022-02-25 10:59:49,982] Trial 0 finished with value: 0.4580152671755725 and parameters: {'booster': 'gbtree', 'colsample_bylevel': 0.9171825663775225, 'colsample_bynode': 0.1444234741048125, 'colsample_bytree': 0.4362117981770428, 'gamma': 7.146184541664092, 'importance_type': 'gain', 'learning_rate': 0.6926919146850347, 'max_delta_step': 2.387598698273216, 'max_depth': 8, 'min_child_weight': 0.7008792624545435, 'n_estimators': 300, 'num_parallel_tree': 6, 'reg_alpha': 9.276318537407738, 'reg_lambda': 0.2521384377872993, 'subsample': 0.19373716199832414}. Best is trial 0 with value: 0.4580152671755725.
[I 2022-02-25 11:00:07,591] Trial 1 finished with value: 0.0 and parameters: {'booster': 'gblinear', 'colsample_bylevel': 0.8067372166937614, 'colsample_bynode': 0.4572763521681874, 'colsample_bytree': 0.816833861079494, 'gamma': 2.282397671950842, 'importance_type': 'cov

KeyboardInterrupt: ignored

In [ ]:
optuna.visualization.plot_param_importances(XGB_study_slow)

In [ ]:
optuna.visualization.plot_optimization_history(XGB_study_slow)

In [ ]:
XGB_slow.fit(X_slow_train, y_slow_train)

XGBClassifier(interaction_constraints='', learning_rate=0.300000012,
              max_depth=6, n_jobs=-1, num_parallel_tree=1, tree_method='exact',
              validate_parameters=1)

In [ ]:
y_slow_prob = XGB_slow.predict_proba(X_slow_test)

In [ ]:
for threshold in range(1,10):
    threshold = round(threshold*0.1, 1)
    y_slow_pred = np.where(y_slow_prob>=threshold, 1, 0)
    print(f'{threshold} :{f1_score(y_slow_test, y_slow_pred[:,1])}')

0.1 :0.6451612903225806
0.2 :0.6976744186046512
0.3 :0.6904761904761905
0.4 :0.7160493827160493
0.5 :0.7272727272727273
0.6 :0.767123287671233
0.7 :0.7714285714285714
0.8 :0.7575757575757577
0.9 :0.7419354838709677


In [ ]:
y_slow_pred_train = XGB_slow.predict(X_slow_train)
f1_score(y_slow_train, y_slow_pred_train)

1.0